# EDA 

### Import Essential Libraries and Dataset

In [5]:
# importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
sb.set()
import warnings
warnings.filterwarnings("ignore")
from collections import Counter

In [13]:
# import dataset
df_people = pd.read_csv('data\\people.csv')
df_sim = pd.read_csv('data\\simulation.csv')

# data information
df_people.info()
df_sim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28957 entries, 0 to 28956
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   action   28957 non-null  object
 1   guest    28957 non-null  bool  
 2   id       28957 non-null  int64 
 3   subject  28957 non-null  object
 4   system   28957 non-null  object
 5   time     28957 non-null  object
dtypes: bool(1), int64(1), object(4)
memory usage: 1.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166941 entries, 0 to 166940
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   action      166941 non-null  object 
 1   direction   57915 non-null   object 
 2   floor       166941 non-null  int64  
 3   subject     166941 non-null  object 
 4   system      166941 non-null  object 
 5   time        166941 non-null  object 
 6   weight      109026 non-null  float64
 7   date        166941 non-null  object 
 8   da

* people.csv documented all of the swipes into the buildings security system
* simulation.csv is a log of each elevator's status and operations, there were total  4 elevators in the 20 story building.